#### Task A
Create a Data Audit for the Pima Indians Diabetes Database. The audit should cover:


    1. Data Overview & Structure:
			- Provide a summary of the dataset.
			- Identify the number of rows and columns.
			- List all variables names and their data types.
			- Classify each variable as categorical or numerical.	

In [25]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools
from sklearn.covariance import EllipticEnvelope



In [8]:
import pandas as pd;
import sweetviz as sv;
data = pd.read_csv('C:\\Users\\v.weber\\Documents\\000 Master Wirtschaftsinformatik FU Berlin\\I\\Applied Analytics\\github stuff\\Applied-Analytics\\data\\diabetes.csv')

In [9]:
rows, cols = data.shape
print(f'rows: {rows}, cols: {cols}')

rows: 768, cols: 9


In [10]:
print(data.dtypes)

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object


    2. Summary Statistics:
			- Compute key statistics such as mean, median, minimum, maximum, and standard deviation.
			- Create a histogram for each variable.
   			- Identify any unusual values that stand out and possible skew of the data.

In [11]:
report = sv.analyze(data);

                                             |                                             | [  0%]   00:00 ->…

In [12]:
report.show_html('report.html')


Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


    3. Outlier:
			- Decide on a formal method to detect univariate outliers.
			- Report the number of outliers per variable.
			- Create scatterplots for each combination of variables.
			- Can you identify possible multivariate outliers that you did not detect as univariate outliers?

In [13]:
def remove_outliers(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    return df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [14]:
data_without_outliers = remove_outliers(data);

In [15]:
report2 = sv.analyze(data)

                                             |                                             | [  0%]   00:00 ->…

In [16]:
report2.show_html('report.html')

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [17]:
def plot_all_scatterplots(df):
    numeric_cols = df.select_dtypes(include=['number']).columns
    combinations = list(itertools.combinations(numeric_cols, 2))
    
    for x, y in combinations:
        plt.figure(figsize=(6, 4))
        sns.scatterplot(data=df, x=x, y=y)
        plt.title(f'Scatterplot of {x} vs {y}')
        plt.show()

In [26]:
outlier_counts = detect_univariate_outliers(data_without_outliers)


NameError: name 'detect_univariate_outliers' is not defined

    5. Correlations.
			- Create a correlation matrix for the dataset.
			- Visualize the correlation matrix.

#### Task B
Based on your Data Audit, prepare your data for further analysis by handling outliers and missing data.


In [23]:
def delete_zeros_except(df, columns):
    df_modified = df.copy()
    cols_to_modify = [col for col in df.columns if col not in columns]
    df_modified[cols_to_modify] = df_modified[cols_to_modify].replace(0, np.nan)
    return df_modified

In [27]:
# delete 0 values
data_with_missing_values = delete_zeros_except(data, ['Pregnancies']);

In [28]:
def hot_deck_imputation(df):
    df_imputed = df.copy()
    for col in df.columns:
        missing_mask = df_imputed[col].isna()
        if missing_mask.any():
            df_imputed.loc[missing_mask, col] = df_imputed[col].dropna().sample(n=missing_mask.sum(), replace=True).values
    return df_imputed

In [29]:
imputated_data = hot_deck_imputation(data_with_missing_values);

In [30]:
df_without_univariate_outliers = remove_outliers(imputated_data);

In [31]:
def remove_multivariate_outliers(df, contamination=0.01):
    numeric_df = df.select_dtypes(include=['number'])
    detector = EllipticEnvelope(contamination=contamination)
    detector.fit(numeric_df)
    mask = detector.predict(numeric_df) == 1
    return df[mask]



In [32]:
df_cleaned_mv = remove_multivariate_outliers(df_without_univariate_outliers)

In [33]:
df_cleaned_mv.to_csv('data_after_monday.csv', index=False)

In [56]:
report = sv.analyze(imputated_data);

                                             |                                             | [  0%]   00:00 ->…

In [57]:
report.show_html('report.html')

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


#### Task C
Test different methods to sample training and test data from the dataset. Compare the sampled datasets based on the descriptive methods you have learned and evaluate which sampling method worked best.

In [66]:
from sklearn.model_selection import train_test_split

# Assuming 'target' is the column indicating diabetes (1 = diabetes, 0 = no diabetes)
X = df_cleaned_mv.drop(columns=['Outcome'])  # Features
y = df_cleaned_mv['Outcome']  # Target variable

# Perform stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)